# Introduction

In [1]:
import libcarna

## Meshes

Using polygonal geometries is useful, for example, to create *markers* or to generally enrich visualizations. In 3D
graphics, *meshes* are used to define polygonal geometries. We start with the definition of a mesh for a *cube*:

In [2]:
cube = libcarna.meshes.create_box(40, 40, 40)

The size of the cube is given in *scene units* (SU), and here it is 40 SU in width, height, and depth. Scene units can
be anything that we agree them to be, like micrometers (e.g., for visualization of cellular image data) or millimeters
(e.g., for image data from computer tomography). It is only important that they are used consistently.

Next, we define some *materials*:

In [3]:
green = libcarna.material('solid', color=libcarna.color.GREEN)
red   = libcarna.material('solid', color=libcarna.color.RED  )

Materials determine how meshes (i.e. polygonal gemetries) are rendered. In LibCarna, a material consists of a *shader*
and a set of *parameters* like colors. Supported shaders comprise `solid` for materials whose colors are affected by
light (the default), and `unshaded` for materials that are colored uniformly.

## Scenes

Now that we have a mesh and some materials in place, we can create a *scene* that defines some spatial relations. We
create two spatial `geometry` objects, both using the `cube` mesh, but with different colors:

In [4]:
GEOMETRY_TYPE_OPAQUE = 1

root = libcarna.node()
libcarna.geometry(
    GEOMETRY_TYPE_OPAQUE,
    parent=root,
    local_transform=libcarna.translate(-10, -10, -40),
    features={
        libcarna.mesh.ROLE_DEFAULT_MESH: cube,
        libcarna.mesh.ROLE_DEFAULT_MATERIAL: green,
    },
)
libcarna.geometry(
    GEOMETRY_TYPE_OPAQUE,
    parent=root,
    local_transform=libcarna.translate(+10, +10, +40),
    features={
        libcarna.mesh.ROLE_DEFAULT_MESH: cube,
        libcarna.mesh.ROLE_DEFAULT_MATERIAL: red,
    },
)

<libcarna._spatial.geometry.<locals>.Geometry at 0x708b45f6a090>

*Scenes* are defined hierarchically, so they form a tree-like structure. The `local_transform` of a node defines the
spatial relation of the node in relation to its parent node. In this example, the green cube is moved by -10 SU along
the x- and y-axes, and by -40 SU along the z-axis. The red cube is moved in the opposite direction.

Note on `GEOMETRY_TYPE_OPAQUE`: A *geometry type* is an arbitrary integer constant, that establishes a relation between
the `geometry` nodes of a scene, and the corresponding rendering stages (see below).

Finally, we define a `camera` that will serve as the point of view for scene rendering:

In [5]:
#camera = libcarna.camera(
#    parent=root,
#    projection=r.frustum(fov=90, z_near=1, z_far=1000),
#    local_transform=libcarna.translate(0, 0, 250),
#)

The `projection` defines the transformation of 3D to planar coordinates. The `frustum` method 